# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E5%9C%8B%E9%9A%9B/"  #國際新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser=webdriver.Chrome(executable_path="chromedriver")

browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height==last_height:
        print("到達頁面底端")
        break
    else:
        print("網頁更新中")
        last_height = new_height

網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 澳洲大火燒毀2座變電所！昨災情「史上最慘」　雪梨近郊飆48.9度高溫紀錄

▲ 消防員在新南威爾斯州莫魯亞滅火。（圖／達志影像／美聯社）
記者陳宛貞／綜合外電報導
澳洲大火本周末進一步惡化，4日下午雪梨郊區氣溫最高達到攝氏48.9度，創下澳洲氣象觀測史上最高紀錄，可能是當天全球最熱的地區。今（5）日東南部部分地區天空被染成血紅色，大量居民趕緊逃命。一名消防員指出，過去24小時是「有史以來最慘的一天」。
根據維多利亞洲吉普斯蘭（Gippsland）環境水土規劃部門，大火連夜在該州奧米歐地區（Omeo）蔓延，形成6000公頃的大火。臨近的新南威爾斯州鄉村消防局指出，首府雪梨以南的臥龍迪利郡（Wollondilly）火勢依然處於失控狀態，近幾個月來已經燒毀264000公頃。
當地天氣狀況4日迅速惡化，由於風力增強，氣溫持續上升，各地都有發生嚴重到極端火災的風險。國家氣象局指出，當天下午首都坎培拉氣溫高達44度，打破80多年來的紀錄；雪梨西部郊區彭里斯（Penrith）氣溫飆升至48.9度，創下整個雪梨盆地的新紀錄。
▲ 相隔6個月的衛星照對比出澳洲大火慘況。（圖／翻攝自推特／@jscarto）
根據新南威爾斯州鄉村消防局統計，該州共發生150起火災，其中64起尚未得到控制，今天出動2300名消防員滅火。局長菲茨西蒙斯（Shane Fitzsimmons）5日在記者會上表示，4日是史上最糟的一天，損失慘重。
一名47歲男子在家中與大火搏鬥時猝死，該州還有4名消防員因吸入濃煙、熱衰竭或燒燙傷住院。菲茨西蒙斯警告，儘管5日氣溫稍稍下降，讓民眾稍稍放鬆，但火勢未來幾天很可能再次惡化。
新南威爾斯州能源部長基恩（Matt Kean）4日在推特上宣布，野火籠罩該州各地，已經燒斷連接該州與維多利亞省的輸電線。根據《法新社》報導，新南威爾斯州有2座變電所被大火燒毀，可能危及最大城雪梨的電力供應。
該州巴特曼斯貝（Batemans Bay）和莫魯亞（Moruya）地區已經有成千上萬人因火災無電可用，但澳洲能源公司Essential Energy表示，部分地區因安全考量無法進入，停電狀況最短得持續一晚。
▲ 志工救出一隻嚴重燒傷的袋貂。（圖／

記者詹雅婷／綜合報導
動脈粥狀硬化（Atherosclerosis，簡稱動脈硬化）經常被認定為「現代人」才有的疾病，是當代飲食、生活方式及環境等因素所綜合起來的產物。但根據美國學者針對16世紀木乃伊的最新研究，即便當時的格陵蘭人以魚類等富含Omega-3脂肪酸的海洋生物為主食，且過著「充滿活力」的生活，但依舊有動脈硬化的問題。
綜合CGTN、Discover等外媒報導，美國心臟科醫師沃恩（L.Samuel Wann）等研究人員利用CT掃描技術，針對4具格陵蘭島因紐特人的木乃伊進行研究。依據骨骼、牙齒來估算，他們是兩名年約18歲至30歲的男子及兩名年約16歲至30歲的女性；另據當時的陪葬品以及服裝來看，他們生活在16世紀，可能住在以石頭、鯨骨、海豹皮建造的棚屋內，以弓箭、長矛獵捕魚類、鳥類、海洋哺乳動物、馴鹿維生。
儘管他們生前的死因不明，但研究結果顯示，當中3具木乃伊的動脈堆積脂肪物質，看起來與動脈硬化患者極為類似。報導指出，由於遺骸不完整，所以動脈硬化的範圍無法做進一步的評估。
動脈硬化被認為是現代人久坐不動、吸菸、飲食習慣不佳所帶來的結果，且過去數十年來，許多人鼓吹攝取富含Omega-3脂肪酸的飲食有助防止動脈硬化。但事實上，導致動脈粥狀硬化的因素有很多，研究認為，除了現代飲食之外，還有其他可能的致病因素，像是古代民族室內生火所產生的環境煙霧等。
▼研究中的木乃伊於1920年代在極冷的環境中被考古學家發現，事後轉移至麻薩諸塞州一間博物館保存。
----------------------------------------------------------------------

[13] 美國各地80場示威「離開中東！」　譴責川普殺害蘇雷曼尼

▲民眾走上街頭抗議，舉著牌子要求美軍停止對伊朗的傷害。（圖／路透）
實習記者施怡妏／綜合報導
成千上萬個反戰團體週六（4日）聚集在全世界各地，譴責美國在巴格達發動的空襲事件，殺害伊朗安全和情報最高指揮官蘇雷曼尼（Qassem Soleimani），包括華府及洛杉磯在內，舉行超過80場的示威活動。
▲示威民眾高喊，「不要更多的炸彈」。（圖／路透）
粉紅代碼領導人本傑明（Medea Benjamin）表示，「這次的示威跟以往不太一樣，參與者多半是年輕人。」數百年示威者在白宮前大喊，「沒有正義就沒有和平，美國離開中東！」

記者張靖榕／綜合外電報導
伊拉克首都巴格達4日發生數起火箭炮攻擊，美國大使館外圍的綠區（Green Zone）受到攻擊外，巴格達以北的美軍基地更遭到數枚火箭炮打入營區內。伊拉克軍方表示，目前無傳出死亡，稍後會進一步更新消息。
路透社報導，數發火箭炮轟炸巴格達慶典廣場以及嘉德里亞（Jadriya），前者距離綠區約5分鐘車程，後者則是8分鐘車程，極有挑釁意味。巴格達北邊薩拉胡丁省（Salahuddin）的美軍巴拉德空軍基地（Balad）則是遭到直接攻擊。
▲▼嘉德里亞社區建物及車輛受創。（圖／翻攝自推特）
《阿拉伯新聞》（Arab News）報導，巴拉德空軍基地內駐有美軍部隊和美國外交官，在轟炸發生之後，營地立刻響起防空警報。
《今日印度》（India Today）指出，綠區附近嘉德里亞的部分建築與路邊車輛遭到火箭炮重創，牆面破洞、鋼骨外露，車輛則是窗戶震碎、車身佈滿碎片孔；該區許多屬於政府建物。美軍基地受到空襲後，直升機隨即升空，在巴格達上空進行巡防預警任務。
▲駐巴格達美國大使館附近，近日成高危險地區。（圖／達志影像／美聯社）
目前尚無法確認襲擊事件，是否與美軍利用無人跡轟炸巴格達國際機場，造成伊朗「聖城軍」（Quds Force）指揮官蘇萊曼尼（Qasem Soleimani）身亡有關，也沒有相關政府或組織出面宣稱發起襲擊。美國政府現在上下氣氛緊繃，擔心伊朗「嚴厲復仇」的宣言會將中東地區情勢更升級，而此刻美國、伊拉克與伊朗三國關係，正處在肅殺氣氛中。
----------------------------------------------------------------------

[23] 跟足球一樣大！疃穆達大王花直徑111cm　整朵「腐屍味」一週爛掉

▲印尼西蘇門答臘雨林中發現111公分長的大王花。（圖／達志影像）
實習記者施怡妏／綜合報導
生態保育人士在印尼西蘇門答臘（West Sumatra）雨林中發現疃穆達大王花（Rafflesia tuan-mudae），花朵直徑長111公分，是史上最大的花朵。自然保育局的艾德（Ade Putra）表示，「這種花的壽命僅有一週，之後便會腐爛。」
印尼蘇門答臘島（Sumatra）阿加姆（Agam）自然保育局的艾德（Ade Putra）表示，「這將是歷史文獻上，有史以來最大朵的疃穆達大王花」，光是花莖的重量就

▲新加坡樟宜機場加強檢疫。（圖／達志影像／美聯社）
記者丁維瑀／綜合報導
新加坡樟宜機場（Changi Airport）自3日晚間開始加強檢疫，針對來自中國湖北省武漢市的旅客，必須接受體溫檢測。新加坡衛生部（MOH）2日表示將繼續密切關注情況，若疑似出現病例，將會送到醫院隔離，展開進一步評估。
武漢近期發生不明原因病毒性肺炎，除了香港衛生署宣布啟動應變計畫，其他國家也紛紛採取措施應對。Yahoo News Singapore報導，來自武漢的旅客必須在新加坡樟宜機場測量體溫，衛生部會持續關注狀況。
衛生建議的海報會張貼於樟宜機場，來自武漢的旅客也會收到相關的指南。新加坡衛生部也提醒所有醫療人員，對有疑似出現肺炎症狀的例子保持警惕，尤其是曾經過武漢的旅客。
《海峽時報》報導，新加坡國家傳染病中心執行主任Leo Yee Sin說明，前往武漢的旅客不需要取消行程，但應該遵從衛生部的指示，持續注意當地的情形。
《英國廣播公司》（BBC）指出，世界衛生組織（WHO）已經注意到病例，正關注最新情況，也與中國政府保持聯繫；發言人賈撒列維克（Tarik Jasarevic）說明正在進行調查，當局還無法證實是什麼病原體導致這些病例。他提到，有數種潛在因素可以導致病毒性肺炎，其中很多都比SARS常見。
報導指出，此次事件讓人們想起18年前的SARS疫情爆發。回到當時，世界衛生組織譴責中國未能徹底報告SARS病例，2002到2003年期間，超過8000人在26個國家中感染這種病毒，在中國造成349人死亡，香港則出現299死。
►武漢肺炎涉及「新型病毒」！香港傳染病專家：相信「已經人傳人」 
►警戒武漢不明肺炎！來台班機登機檢疫6人有上呼吸道症狀
----------------------------------------------------------------------

[35] 澳洲野火濃煙驚見「惡魔臉」…如準備吞噬大地　火勢恐惡化

記者張方瑀／綜合報導
澳洲新南威爾斯州（New South Wales）野火已經延燒數月，造成至少20人死亡、28人失蹤，還有4.3億隻動物死亡。一名住在維多利亞州的農夫，就拍下一張烈焰燒的照片，驚人的是，在大火中居然出現一張「惡魔臉」，讓他感嘆，現在澳洲的慘況，真的是正在被惡魔踐踏。
住在維多利亞州的農夫卡爾佛特（Craig Calver

▲TOYOTA日前向美國專利商標局申請了全新SUV車名「4ACTIVE」的商標權，圖為FT-4X Concept。（圖／翻攝自TOYOTA，以下同）
記者游鎧丞／綜合報導
之前美國TOYOTA高層Jack Hollis便曾表態，原廠有意在C-HR此一級距，另行推出一款專職「越野」的全新跨界SUV；無獨有偶的是，TOYOTA日前就向美國專利商標局註冊了「4ACTIVE」全新商標；外界一般推測，4ACTIVE很有可能就是TOYOTA意欲在美國阿拉巴馬產線全新生產的小型跨界SUV。
▲4ACTIVE很有可能將會立基於概念車FT-4X Concept的基礎開發，圖為FT-4X Concept。
TOYOTA預計將在2023年之前，推出多達12款的新車（包含小改款車型），當中最受矚目的，莫過於計畫於C-HR同級的全新跨界SUV；雖說目前官方並未證實之前註冊的4ACTIVE是否就是這款全新的戰略跨界SUV，但知情人士透露，位於美國阿拉巴馬州的Huntsville產線，已著手準備4ACTIVE的相關生產作業。
4ACTIVE之所以如此備受期待，很大一部分在於TOYOTA高層日前便曾透露，將在C-HR此一級距，推出另一款車身尺碼更大，更具實用性的車款，同時一大亮點，莫過於這款新車將是TOYOTA在該級距當中，首度叩關越野性能的全新戰略作品；同時受惠於TNGA-C模組化底盤平台，換取更具餘裕的車室空間。
▲這款4ACTIVE是否就會是早先和泰外傳將會國產化的全新跨界SUV？目前仍是未知數，圖為FT-4X Concept。
不僅國外車媒相當關注，就連台灣車市也可說是相當注意4ACTIVE這台新車；畢竟早先台灣總代理和泰汽車就曾表示，將在2020年推出一款國產化的小型跨界SUV；在早先亮相的日規Raize已被打槍之後，4ACTIVE便是近期最為符合該項設定的全新車款；倘若一如外媒所言，4ACTIVE將借重FT-4X Concept基礎所開發而來，再加上越野的車格設定，未來也不排除將成為Suzuki Jimny的首要勁敵。
不過一切都還只是外界在各自表述就是了…。
----------------------------------------------------------------------

[47] 台灣之光「福灣巧克力」列中國品牌　台日網友怒轟日本阪急：徹底謝罪

▲台灣

記者張方瑀／綜合報導
美軍3日無預警空襲擊殺伊朗「聖城軍」指揮官蘇萊曼尼（Qassem Soleimani）。BBC指出，英國首相強生對於這場空襲完全不知情，嚴重影響英國在中東地區400名士兵的安全。保守黨議員更痛批，所謂聯盟應該是給敵人驚喜，而不是給盟友。
目前英國在中東地區仍有約400名駐軍，但BBC報導指出，在美國進行空襲之前，強生對於此事一無所知。英國工黨領袖柯賓（Jeremy Corbyn）立刻詢問強生，在攻擊之前到底知道多少，並在信中提到，若是英國知道此事，是否會表達反對的立場。柯賓也要求召開緊急會議，討論這場空襲可能造成的後果，並確保政府採取正確措施保護英國公民的安全。
保守黨議員圖根達特（Tom Tugendhat）表示，現今的「白宮模式」就是不和盟友分享訊息，這是個非常令人擔憂的情況，「我一直認為，擁有盟友的目的，就是讓我們可以一起給敵人驚喜，而不是互相給對方帶來驚喜。」
英國外交大臣拉布（Dominic Raab）已經與美國國務卿蓬佩奧（Mike Pompeo）通話，清楚表達英國的立場，並發聲明希望雙方緩和緊張局勢，「英國承認蘇萊曼尼確實充滿威脅，但進一步的衝突並不符合英國利益。」
----------------------------------------------------------------------

[59] 三星率先在CES 2020展前　發表Galaxy S10 Lite與Note10 Lite

▲三星率先在CES 2020展前發表兩款新機。（圖／三星提供）
記者姚惠茹／台北報導
全球最大消費性電子展CES 2020今年1月7日即將在美國拉斯維加斯開展，三星電子（SAMSUNG）宣布在展前發表Galaxy S10 Lite與Galaxy Note10 Lite兩款新機，主打以實惠的價格就能擁有最新的相機技術、S Pen、顯示技術和超長續航力電池。
三星這次在Galaxy S10 Lite和Galaxy Note10 Lite搭載多項旗艦級功能，包括6.7吋的無邊際Infinity-O極限全螢幕，搭載4,500mAh大容量電池，具備閃電快充功能，並串聯三星生態圈中的APP及服務，包含Bixby、Samsung Pay、Samsung Health等，受到Samsung Knox的國防級隱私防護。
▲三星Galaxy S

▲埼玉縣川口市3日晚間火災。（示意圖／取自Pixabay）
記者林瑩真／編譯
日本埼玉縣川口市3日晚間7時許，一間木造的2層建築起火，消防隊出動18輛消防車、花費3小時才全面撲滅火勢。然而，34歲的長女難得因過年返家探望父親，卻送醫不治。
綜合日媒報導，位於川口市安行領根岸附近的2層樓民宅，在晚間7時45分起火，當晚有超過10名親戚聚在一起慶祝過年，發現火勢後立刻向消防隊報案，隨後逃離住宅，全員無傷。
消防隊出動18輛消防車，並花費約3小時撲滅火勢，不過找到大女兒水上未來時，她已經全身嚴重燒傷，送醫急救仍不治。
▼影片取自YouTube，如遭刪除請見諒。
另外，消防隊也連絡不上65歲的父親山田雅廣，根據親戚說法，起火當時兩人都在臥室中睡覺。由於現場還有一具燒焦到無法辨認身分的遺體，目前警方正在確認是否就是山田雅廣，以及起火原因。
報導指出，山田雅廣平時跟妻子2人一起生活，是恰逢新年，有大約10名親戚前來聚會，大女兒水上未來也好不容易從岐阜縣回來探望，沒想到卻從此天人永隔。
----------------------------------------------------------------------

[68] 色情網站賠22名女大生3.8億！　曝誘騙手法「私密影片親朋好友全看過」

▲聖地牙哥色情產業黑暗面，隨著受害女性提出集體訴訟，再次受到社會關注。（圖／pixabay）
記者張靖榕／綜合外電報導
美國色情產業地上地下蓬勃發展，而其黑暗面也長期為人詬病。近日一項案件中，22名不願對外具名的女性對色情網站「GirlsDoPorn」提出集體訴訟，表示各自在不知情下，被一家聖地牙哥（San Diego）假冒的模特兒經紀公司誘導拍攝性愛影片，並上傳到各大成人網站；法官判定罪證成立，「GirlsDoPorn」必須賠償1270萬美元（約新台幣3.815億元）給22名受害女性。
英國《衛報》報導，在西方文化中，不知名或是匿名的人，男性通常以「John Do」表示，女性則以「Jane Doe」稱之，而在此集體訴訟案中，22名不願具名的女性便以「Jane Doe」1號到「Jane Doe」22號稱呼。
「GirlsDoPorn」於2006年上線，由來自紐西蘭的普萊特（Michael Pratt）創立，交給賈西亞（Ruben Garcia）和沃爾夫（Matthew Wo

國際中心／綜合報導
來自台東的阿美族男子陳炯明17歲跑船時，因與父親起爭執，被「丟包」西班牙偏遠離島，沒想到這一待就是38年，如今他在移民署台東服務站專員陳允萍的幫助之下，終於辦妥文件，有望回台灣。他接受專訪時表示，並不恨父親把自己丟在海外，只怪自己當初沒盡孝道。
回憶起被留在陌生小島上的那一晚，55歲的陳炯明表示，當時自己因為跟當地人喝酒醉倒，醒來趕到碼頭發現船已離港，才驚覺自己「慘了」。後來他過了一段流浪街頭的日子後靠著做「黑工」度日，但有些雇主怕用非法勞工被罰，幸好他到處去問，碰到好心人願意讓他靠洗碗賺點錢。
陳炯明提到，「我想辦法到（蘭薩羅特島首府）阿雷西費（Arrecife）後，碰到台灣飯店老闆，聊天後就問能不能給我工作，剛好對方需要員工，就開始上班，工作一、兩年，就認識現在的老婆了。」
不過違法的高風險讓陳炯明有段時間得時常換工作，最慘曾經失業一、兩年都沒有零工可打。他表示，連老婆都沒有上班那陣子，全家唯一的收入只有育兒補助跟失業津貼，「那時多半靠當地認識的華人朋友，借點小錢、幫點忙，苦日子就這樣過著過著，就過去了。」
目前陳炯明還在同家飯店工作，但老闆已經換人，為了養活6子1女，他幾乎全年無休，他有兩個還在念小學的孩子，最大的兒子則已過18歲，進入大學讀書。
►「滯留西班牙小島39年」陳炯明取得回台文件　可望年前返鄉
----------------------------------------------------------------------

[79] 影／美軍擊斃「伊朗007」川普：早就該除掉了　增派3500人部署中東

記者張靖榕／綜合外電報導
美國將伊朗少將菁英部隊「聖城軍」指揮官蘇萊曼尼（Qasem Soleimani）擊殺後，面對中東情勢陡升，已經增派至少3000人部隊。總統川普才看似履行了選前政見，讓美國大兵回家，如今又派兵重回老地方，但對於造成情勢轉折毫不後悔，「他（蘇萊曼尼）好多年前早就該被除掉了」。
▲伊朗少將指揮官蘇萊曼尼（Qasem Soleimani）在巴格達國際機場被美軍空襲身亡。（圖／達志影像／美聯社）
綜合外電報導，美國五角大廈已經派遣3000到3500名士兵赴中東地區，為美軍及相關設施人員，可能遭到伊朗任何形式的報復做準備。2名國防部官員指出，最新部署的部隊為美軍第82空降師（82nd Airborne 

國際中心／綜合報導
伊拉克當地時間周五(3日)淩晨，美軍在巴格達國際機場附近發動空襲，4枚精密導彈擊中兩輛座車，造成伊朗「聖城旅」指揮官蘇萊曼尼(Qassem Soleimani)少將身亡。
外媒隨後披露了美軍這次行動的細節，指出蘇萊曼尼當時甫從敘利亞抵達巴格達機場。他的座車離開機場後不久，即被美軍精準的導彈擊中。
《每日郵報》報導，蘇萊曼尼在3日淩晨剛剛從敘利亞的大馬士革搭機來到巴格達。伊拉克的民兵組織「人民動員組織」副指揮官穆罕迪斯(Abu Mahdi al-Muhandis)當時親往機場接機。
美國一名不願透露姓名的官員表示，當蘇萊曼尼和他隨行人員分別乘坐的兩輛黑色轎車正在巴格達機場附近的一條通路上行駛時，被美軍「MQ-9收割者無人機」(MQ-9 Reaper drone)所發射的4枚導彈擊中。  除了蘇萊曼尼，這次攻擊同時導致了「人民動員組織」副指揮官穆罕迪斯等另外4人死亡。
「人民動員組織」的官員表示，蘇萊曼尼遇襲後被炸成碎片。他們是透殘肢上所戴的一枚大戒指辨認出他的身分。至於穆罕迪斯的屍體，該組織稱還目前沒有找到。
當地的民兵指揮官侯賽尼告訴「路透社」說，蘇萊曼尼跟穆罕迪斯同坐一輛車，這輛車被兩枚導彈擊中，另外一輛車遭遇相同命運。侯賽尼說：「美國的罪犯早一步掌握了車子移動的詳細資訊。」
美國國防部表示，總統川普在美國時間2日上午批准了這次針對蘇萊曼尼的無人機攻擊，並稱這項格殺令是「為了阻止伊朗未來的攻擊」。
伊朗對此事迅速回應。外長札里夫在社群媒體上稱，謀殺蘇萊曼尼是「國際恐怖主義行為」，並且愚蠢地加劇了緊張局勢。伊朗最高領袖哈米尼則表示，殺害蘇萊曼尼的「罪犯」會遭到嚴厲的報復。
----------------------------------------------------------------------

[90] 美國、日本等26國為黑鷹殉職將士哀悼　外交部表達感謝

▲外交部長吳釗燮弔唁黑鷹失事罹難者。（圖／記者林敬旻攝）
記者蔣婕妤／台北報導
國軍UH-60M黑鷹直升機2日在新北市烏來山區失事，機上包括參謀總長沈一鳴上將等8位將士不幸因公殉職。包含美國、日本等26個國家對此向我國表達哀悼之意，外交部也對此表示感謝。
外交部長吳釗燮於3日上午率三位次長及相關主管向殉職的8位將士表達崇高敬意，也向家屬致上深切哀悼，吳釗燮也為其他5位受傷

In [6]:
browser.quit()